# Análisis de viajes en taxi – Zuber 

## Introducción

Zuber, una nueva empresa de viajes compartidos, quiere analizar los viajes en taxi realizados en Chicago durante noviembre de 2017 para entender:
- Las empresas de taxi más activas.
- Los barrios con mayor número de llegadas.
- El impacto del clima en la duración de los viajes.

Este análisis se basa en datos extraídos de una base de datos y archivos CSV proporcionados. Además, se probará una hipótesis sobre cómo las condiciones climáticas afectan los viajes hacia el aeropuerto en un día específico.

---

## Tabla de contenidos

1. [Carga y revisión de datos](#carga-y-revisión-de-datos)
2. [Análisis de barrios por viajes finalizados](#análisis-de-barrios-por-viajes-finalizados)
3. [Análisis de empresas de taxi](#análisis-de-empresas-de-taxi)
4. [Gráficos de resultados](#gráficos-de-resultados)
5. [Prueba de hipótesis](#prueba-de-hipótesis)
6. [Conclusiones](#conclusiones)

### 1. Carga y revisión de datos

Importamos los archivos CSV con información sobre las empresas de taxis y los barrios donde finalizaron los viajes. Estos datos fueron obtenidos previamente mediante consultas SQL.

Antes de analizarlos, verificamos que:

• Los archivos se hayan cargado correctamente.

• Las columnas tengan los tipos de datos adecuados.

• No haya valores nulos que interfieran con el análisis.

Esta revisión nos permite asegurarnos de que los DataFrames están listos para aplicar filtros, agrupaciones y visualizaciones.

In [ ]:
import pandas as pd

# Importamos los datasets
df_companies = pd.read_csv('../Data/moved_project_sql_result_01.csv')
df_neighborhoods = pd.read_csv('../Data/moved_project_sql_result_04.csv')

# Mostrar información general sobre los DataFrames
df_companies.info()
print()
df_neighborhoods.info()

### 2. Análisis de barrios por viajes finalizados

Los datos están completos y tienen el tipo de dato correcto. En este paso vamos a identificar los **10 barrios de Chicago con mayor número promedio de viajes finalizados** durante noviembre de 2017. Para eso, usamos el archivo 'project_sql_result_04.csv', que contiene:

 - **dropoff_location_name:** nombre del barrio donde terminó el viaje

 - **average_trips:** promedio de viajes finalizados por barrio

Este análisis es importante para conocer las **zonas de mayor demanda**, lo cual puede dar pistas útiles para **optimización de flotas, campañas de marketing o diseño de rutas estratégicas**.

Vamos a ordenar los datos de forma descendente por número de viajes promedio y luego seleccionar los primeros diez.

In [ ]:
# Identificamos los 10 barrios principales por finalización del recorrido
top_neighborhoods = df_neighborhoods.sort_values(by='average_trips', ascending=False).head(10)

# Mostramos el resultado
top_neighborhoods

### 3. Análisis de empresas de taxi

Ahora vamos a identificar las **10 empresas de taxis con mayor número de viajes registrados** durante los días 15 y 16 de noviembre de 2017. Usaremos el archivo `'project_sql_result_01.csv'`, que contiene:

- `company_name`: nombre de la empresa de taxis  
- `trips_amount`: número total de viajes realizados por la empresa el 15 y 16 de noviembre de 2017

Este análisis nos permite conocer qué empresas tuvieron mayor participación en el mercado durante esos días. Con estos datos, podemos **observar tendencias** que podrían estar relacionadas con cobertura, reputación o promociones específicas.

Vamos a ordenar los datos en orden descendente por número de viajes y luego seleccionaremos las 10 primeras empresas.

In [ ]:
# Identificamos las 10 empresas con más viajes
top_companies = df_companies.sort_values(by='trips_amount', ascending=False).head(10)

# Mostramos el resultado
top_companies

### 4. Gráficos de resultados

Ya que identificamos los 10 barrios con mayor número promedio de viajes finalizados y las 10 empresas de taxis con mayor número de viajes durante los días 15 y 16 de noviembre, ahora vamos a visualizar estos datos con gráficos de barras para comparar de forma clara y rápida los valores, detectar patrones y facilitar la interpretación de los datos. En este caso:

- La primera gráfica mostrará los **barrios más populares como destino final**.
- La segunda gráfica mostrará las **empresas de taxis más activas** en esos días.

In [ ]:
import matplotlib.pyplot as plt

# Gráfica 1: Barrios más populares
plt.figure(figsize=(10, 5))
plt.bar(top_neighborhoods['dropoff_location_name'], top_neighborhoods['average_trips'], color='steelblue')
plt.title('Top 10 barrios por promedio de viajes finalizados')
plt.xlabel('Barrio')
plt.ylabel('Promedio de viajes finalizados')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# Gráfica 2: Empresas más activas
plt.figure(figsize=(10, 5))
plt.bar(top_companies['company_name'], top_companies['trips_amount'], color='orange')
plt.title('Top 10 empresas de taxi por número de viajes')
plt.xlabel('Empresa')
plt.ylabel('Número de viajes')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

#### Conclusión parcial

Los gráficos permiten visualizar más facilmente los datos obtenidos:

- **Loop**, **River North** y **Streeterville** destacan como los barrios más populares como destino final de los viajes.
- **Flash Cab** y **Taxi Affiliation Services** fueron las empresas con mayor número de viajes registrados durante los días 15 y 16 de noviembre de 2017.

Con esta información identificamos zonas de alta concentración de viajes y empresas líderes en volumen de servicio. Esto será útil más adelante para interpretar si el impacto del clima afecta de alguna forma diferente a ciertos factores.

### 5. Prueba de hipótesis

Vamos a analizar si las condiciones climáticas afectan la duración de los viajes en taxi desde **Loop** hacia el **Aeropuerto O’Hare**, específicamente los **sábados** de noviembre de 2017.

> Ya filtramos estos viajes y los combinamos con los datos del clima en pasos anteriores. Ahora vamos a trabajar directamente con esa tabla (`project_sql_result_07.csv`), que ya incluye el campo `weather_conditions`.

#### Objetivo

Determinar si existe una diferencia significativa en la duración promedio de los viajes entre días con **buen clima (Good)** y **mal clima (Bad)**.

#### Hipótesis

- **Hipótesis nula (H₀):** No hay diferencia en la duración promedio entre climas buenos y malos.
- **Hipótesis alternativa (H₁):** Sí hay una diferencia significativa en la duración promedio entre climas buenos y malos.

#### Metodología

1. Cargamos los datos de viajes ya filtrados y clasificados por clima.
2. Dividimos la duración de los viajes en dos grupos según `weather_conditions`.
3. Revisamos estadísticas descriptivas de cada grupo (mínimo, máximo, media, etc.).
4. Aplicamos una **prueba de hipótesis (t-test de dos muestras)** para determinar si la diferencia en duraciones es estadísticamente significativa.

---

Este análisis nos ayuda a identificar si el clima tiene un impacto tangible en los tiempos de traslado, lo cual puede ser útil para planificación operativa, estimaciones de viaje y decisiones estratégicas para empresas de transporte.

In [ ]:
from scipy import stats

# 1. Cargar el archivo y revisar su estructura.
df = pd.read_csv('../Data/moved_project_sql_result_07.csv')

# Revisar si hay valores ausentes que puedan afectar
print(df.isna().sum())

# Separar los viajes por clima
good_weather = df[df['weather_conditions'] == 'Good']['duration_seconds']
bad_weather = df[df['weather_conditions'] == 'Bad']['duration_seconds']

In [ ]:
# 2. Análisis descriptivo
# Revisar estadísticas básicas de cada grupo
print("Viajes con buen clima:")
print(good_weather.describe())

print("\nViajes con mal clima:")
print(bad_weather.describe())

In [ ]:
# 3. Prueba estadística e interpretación

# Prueba de hipótesis: t-test
alpha = 0.05
t_stat, p_val = stats.ttest_ind(good_weather, bad_weather, equal_var=False)

print("\nResultados del t-test:")
print()
print("Estadístico t:", t_stat)
print("p-valor:", p_val)


# Interpretación
if p_val < alpha:
    print("Rechazamos la hipótesis nula. El clima afecta significativamente la duración de los viajes.")
else:
    print("No se puede rechazar la hipótesis nula. No hay evidencia suficiente de que el clima afecte la duración.")

### 6. Conclusiones y siguientes pasos

#### Conclusión del análisis

Durante noviembre de 2017, encontramos que:

- Los **barrios con más viajes finalizados** fueron Loop, River North y Streeterville, lo que sugiere zonas de alta atracción (por trabajo, turismo o conexión).
- Las **empresas más activas** fueron Flash Cab y Taxi Affiliation Services.
- Al comparar viajes desde Loop al Aeropuerto O’Hare en sábados, descubrimos que **las condiciones climáticas sí afectan significativamente la duración del trayecto**.

> En promedio, un viaje en clima malo dura **21.3% más** que uno en clima bueno (de 1999 a 2427 segundos).

Esto sugiere que el clima debe considerarse al **planear tiempos estimados de viaje** o **optimizar rutas**.

---

#### Siguientes pasos sugeridos

1. **Modelos predictivos**: Incorporar variables como clima, hora y día para anticipar la duración de los viajes con mayor precisión.

2. **Optimización de flota**: Aunque inicialmente se pensó en reforzar destinos con más finalizaciones, sería más útil ubicar taxis donde **empiezan los viajes**, ya que eso activa la demanda.

3. **Análisis de la competencia**: Observar el comportamiento de las empresas más activas (Flash Cab, etc.) para identificar si operan con mejor cobertura, tarifas, reputación u horarios específicos.

4. **Identificación de corredores estratégicos**:
   - Analizar los tramos más frecuentes entre barrios (por ejemplo, Loop → O’Hare).
   - Implementar **estrategias de marketing específicas** o **rutas compartidas con tarifa reducida** en estos corredores.
     - Ejemplo: promociones grupales o servicios tipo shuttle en rutas de alta demanda para mejorar rentabilidad por kilómetro.